<a href="https://www.kaggle.com/code/mafiosoquasar/time-series-sales-forecasting-with-pytorch?scriptVersionId=211040621" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [179]:
import argparse
import os
import random,numpy
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

In [180]:
pip install pandas 


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [181]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torchvision.transforms.functional as RF
from torchvision.models.feature_extraction import create_feature_extractor
from PIL import Image
import numpy as np
import random,cv2,pandas,os,numpy
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from sklearn.preprocessing import OneHotEncoder

In [182]:
seed = 999
print("Random Seed: ", seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

nz = 100
beta1 = 0.5
lr = 0.0001
batch_size=1000
ngpu=1
ngf,nc = 3,3
ndf = 64
shape=(308263,10)

device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

Random Seed:  999


In [183]:
train = pandas.read_csv('/kaggle/input/rohlik-sales-forecasting-challenge-v2/sales_train.csv', parse_dates=['date'])
test = pandas.read_csv('/kaggle/input/rohlik-sales-forecasting-challenge-v2/sales_test.csv', parse_dates=['date'])
ss = pandas.read_csv('/kaggle/input/rohlik-sales-forecasting-challenge-v2/solution.csv')

inventory = pandas.read_csv('/kaggle/input/rohlik-sales-forecasting-challenge-v2/inventory.csv')
weights = pandas.read_csv('/kaggle/input/rohlik-sales-forecasting-challenge-v2/test_weights.csv')
calendar = pandas.read_csv('/kaggle/input/rohlik-sales-forecasting-challenge-v2/calendar.csv', parse_dates=['date'])

Frankfurt_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Frankfurt_1"')
Prague_2 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Prague_2"')
Brno_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Brno_1"')
Munich_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Munich_1"')
Prague_3 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Prague_3"')
Prague_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Prague_1"')
Budapest_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Budapest_1"')

def process_calendar(df):

    df = df.sort_values('date').reset_index(drop=True)

    df['next_holiday_date'] = df.loc[df['holiday'] == 1, 'date'].shift(-1)
    df['next_holiday_date'] = df['next_holiday_date'].bfill()
    df['days_to_holiday'] = (df['next_holiday_date'] - df['date']).dt.days
    df.drop(columns=['next_holiday_date'], inplace=True)

    df['next_shops_closed_date'] = df.loc[df['shops_closed'] == 1, 'date'].shift(-1)
    df['next_shops_closed_date'] = df['next_shops_closed_date'].bfill()
    df['days_to_shops_closed'] = (df['next_shops_closed_date'] - df['date']).dt.days
    df.drop(columns=['next_shops_closed_date'], inplace=True)

    df['day_after_closing'] = (
        (df['shops_closed'] == 0) & (df['shops_closed'].shift(1) == 1)
    ).astype(int)

    df['long_weekend'] = (
        (df['shops_closed'] == 1) & (df['shops_closed'].shift(1) == 1)
    ).astype(int)

    df['weekday'] = df['date'].dt.weekday 
    return df

dfs = ['Frankfurt_1', 'Prague_2', 'Brno_1', 'Munich_1', 'Prague_3', 'Prague_1', 'Budapest_1']

processed_dfs = [process_calendar(globals()[df]) for df in dfs]

calendar_extended = pandas.concat(processed_dfs).sort_values('date').reset_index(drop=True)

train_calendar = train.merge(calendar_extended, on=['date', 'warehouse'], how='left')
train_inventory = train_calendar.merge(inventory, on=['unique_id', 'warehouse'], how='left')
train_data = train_inventory.merge(weights, on=['unique_id'], how='left')

test_calendar = test.merge(calendar_extended, on=['date', 'warehouse'], how='left')

test_data = test_calendar.merge(inventory, on=['unique_id', 'warehouse'], how='left')
test_data['YEAR'] = test_data['date'].dt.year
test_data['MONTH'] = test_data['date'].dt.month
test_data['DAY'] = test_data['date'].dt.day
test_data = test_data.drop(columns=['date'])

train_data.dropna(subset=['sales'], inplace=True)
train_data = train_data.sort_values(['unique_id', 'date'])

train_data.head()

train_data['YEAR'] = train_data['date'].dt.year
train_data['MONTH'] = train_data['date'].dt.month
train_data['DAY'] = train_data['date'].dt.day
train_data_ = train_data.drop(columns=['date'])
train_data_ = train_data.drop(columns=['availability'])
train_data_ = train_data_.drop(columns=['weight'])
x = train_data_.drop(columns=['sales'])

x.columns, test_data.columns

(Index(['unique_id', 'date', 'warehouse', 'total_orders', 'sell_price_main',
        'type_0_discount', 'type_1_discount', 'type_2_discount',
        'type_3_discount', 'type_4_discount', 'type_5_discount',
        'type_6_discount', 'holiday_name', 'holiday', 'shops_closed',
        'winter_school_holidays', 'school_holidays', 'days_to_holiday',
        'days_to_shops_closed', 'day_after_closing', 'long_weekend', 'weekday',
        'product_unique_id', 'name', 'L1_category_name_en',
        'L2_category_name_en', 'L3_category_name_en', 'L4_category_name_en',
        'YEAR', 'MONTH', 'DAY'],
       dtype='object'),
 Index(['unique_id', 'warehouse', 'total_orders', 'sell_price_main',
        'type_0_discount', 'type_1_discount', 'type_2_discount',
        'type_3_discount', 'type_4_discount', 'type_5_discount',
        'type_6_discount', 'holiday_name', 'holiday', 'shops_closed',
        'winter_school_holidays', 'school_holidays', 'days_to_holiday',
        'days_to_shops_closed', 'day

In [184]:
x = x[['unique_id', 'YEAR', 'MONTH', 'DAY', 'total_orders', 'sell_price_main',
        'type_0_discount', 'type_1_discount', 'type_2_discount',
        'type_3_discount', 'type_4_discount', 'type_5_discount',
        'type_6_discount', 'holiday', 'shops_closed',
        'winter_school_holidays', 'school_holidays', 'days_to_holiday',
        'days_to_shops_closed', 'day_after_closing', 'long_weekend', 'weekday',
        'product_unique_id']]
test_data = test_data[['unique_id', 'YEAR', 'MONTH', 'DAY', 'total_orders', 'sell_price_main',
        'type_0_discount', 'type_1_discount', 'type_2_discount',
        'type_3_discount', 'type_4_discount', 'type_5_discount',
        'type_6_discount', 'holiday', 'shops_closed',
        'winter_school_holidays', 'school_holidays', 'days_to_holiday',
        'days_to_shops_closed', 'day_after_closing', 'long_weekend', 'weekday',
        'product_unique_id']]
test_data.head()

,unique_id,YEAR,MONTH,DAY,total_orders,sell_price_main,type_0_discount,type_1_discount,type_2_discount,type_3_discount,...,holiday,shops_closed,winter_school_holidays,school_holidays,days_to_holiday,days_to_shops_closed,day_after_closing,long_weekend,weekday,product_unique_id
0,1226,2024,6,3,8679.0,13.13,0.00000,0.0,0.0,0.0,...,0,0,0,0,33.0,147.0,0,0,0,627
1,1226,2024,6,11,8795.0,13.13,0.15873,0.0,0.0,0.0,...,0,0,0,0,25.0,139.0,0,0,1,627
2,1226,2024,6,13,10009.0,13.13,0.15873,0.0,0.0,0.0,...,0,0,0,0,23.0,137.0,0,0,3,627
3,1226,2024,6,15,8482.0,13.13,0.15873,0.0,0.0,0.0,...,0,0,0,0,21.0,135.0,0,0,5,627
4,1226,2024,6,9,8195.0,13.13,0.00000,0.0,0.0,0.0,...,0,0,0,0,27.0,141.0,0,0,6,627


categorical_columns = x.select_dtypes(include=['object']).columns.tolist()
encoder = OneHotEncoder(sparse_output=False)
one_hot_encoded = encoder.fit_transform(x[categorical_columns])

categorical_columns_ = test_data.select_dtypes(include=['object']).columns.tolist()
sub_ = encoder.fit_transform(test_data[categorical_columns_])

In [185]:
x_ = torch.from_numpy(x.to_numpy()).type('torch.FloatTensor')
y_ = torch.from_numpy(train_data_['sales'].to_numpy()).type('torch.FloatTensor')
sub = torch.from_numpy(test_data.to_numpy()).type('torch.FloatTensor')

In [186]:
x_.shape, sub.shape

(torch.Size([4007367, 23]), torch.Size([47021, 23]))

In [187]:
train_x = torch.utils.data.DataLoader(x_,batch_size=batch_size)
train_y = torch.utils.data.DataLoader(y_,batch_size=batch_size)
test_sub = torch.utils.data.DataLoader(sub,batch_size=batch_size)

In [188]:
def weights_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight.data, gain=0.02)
        
class RF_NET(nn.Module):
    def __init__(self):
        super().__init__()
        self.rafire = nn.Sequential(
            
            nn.Linear(23, 1524),
            nn.BatchNorm1d(1524),
            nn.ReLU(),
            
            nn.Linear(1524, 824),
            nn.BatchNorm1d(824),
            nn.ReLU(),
            
            nn.Linear(824, 424),
            nn.BatchNorm1d(424),
            nn.ReLU(),
            
            nn.Linear(424, 212),
            nn.BatchNorm1d(212),
            nn.ReLU(),
            
            nn.Linear(212, 124),
            nn.BatchNorm1d(124),
            nn.ReLU(),
            
            nn.Linear(124, 1)
        )

    def forward(self,x):
        #x,_=self.rafire_0(x)
        #print(x,x.shape)
        return self.rafire(x)

In [189]:
EFF_NET = RF_NET().float()
EFF_NET= nn.DataParallel(EFF_NET).to(device)

In [190]:
criterion = nn.L1Loss()

optimizer = optim.AdamW(EFF_NET.parameters(), lr=lr,betas=(beta1, 0.999))
scheduler=torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.86)

In [ ]:
high_rf,i,j_r,k,z_w_=100000,0,0,0,[]
    
correct,total=0,0

while(True):
    optimizer.zero_grad()
    for data,label in zip(train_x,train_y):
        #z_ = nn.Embedding(10, 3, dtype=torch.float64)
        #print(z_(torch.FloatTensor(data)))
        output = EFF_NET(data.to(device)).view(-1)
        err_real = criterion(output, label.to(device))
        err_real.backward()
        optimizer.step()
        #print(err_real.item())
        
    print(f"EPOCH : {i} LOSS_wl : {err_real.item()}")

    if len(z_w_)>=3:
        if len([True for i in range(1,4) if z_w_[len(z_w_)-i]<=z_w_[len(z_w_)-3] and z_w_[len(z_w_)-i]>=z_w_[len(z_w_)-4]])==3:
            z_w_,j_r=[],0
            print(f"lr_br:{optimizer.param_groups[0]['lr']}".upper())
            scheduler.step()
            print(f"lr_up:{optimizer.param_groups[0]['lr']}".upper())
            
    z_w_.append(err_real.item())
    if err_real.item()<high_rf:
            high_rf=err_real.item()
            torch.save(EFF_NET.state_dict(),f'/kaggle/working/{err_real.item()}.pth')
    if i==3:
        break
    i+=1

EPOCH : 0 LOSS_wl : 52.98769760131836
